# Yoked Crop Visualization

This notebook visualizes crop parameters to understand how **yoked crops** work.

**Yoked crops** share the same random seed, which means:
- They sample from the same random number sequence
- With standard RRC, they attempt the same (scale, ratio, position) draws
- Different scale ranges produce **zoom pairs** centered on the same region

We visualize crop regions as rectangles overlaid on a plot where axes represent image dimensions.

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from slipstream.decoders.numba_decoder import (
    _generate_random_crop_params_batch,
    _generate_direct_random_crop_params_batch,
)

In [ ]:
def get_crop_params(width, height, scale, ratio, seed, crop_mode="standard"):
    """Generate crop parameters for a single image.
    
    Returns: (x, y, crop_w, crop_h)
    """
    widths = np.array([width], dtype=np.int32)
    heights = np.array([height], dtype=np.int32)
    log_r_min = math.log(ratio[0])
    log_r_max = math.log(ratio[1])
    
    if crop_mode == "standard":
        params = _generate_random_crop_params_batch(
            widths, heights, scale[0], scale[1], log_r_min, log_r_max, seed
        )
    else:
        params = _generate_direct_random_crop_params_batch(
            widths, heights, scale[0], scale[1], log_r_min, log_r_max, seed
        )
    return params[0]  # (x, y, w, h)

In [ ]:
def plot_crop_regions(ax, width, height, crops, title=""):
    """Plot crop regions as rectangles on axes sized to image dimensions.
    
    Args:
        ax: matplotlib axis
        width, height: image dimensions
        crops: list of (name, x, y, w, h, color) tuples
        title: plot title
    """
    ax.set_xlim(0, width)
    ax.set_ylim(height, 0)  # Flip y-axis (image coords)
    ax.set_aspect('equal')
    ax.set_xlabel('x (pixels)')
    ax.set_ylabel('y (pixels)')
    
    # Draw image boundary
    ax.add_patch(patches.Rectangle(
        (0, 0), width, height, 
        linewidth=2, edgecolor='black', facecolor='lightgray', alpha=0.3
    ))
    
    # Draw crops
    for name, x, y, w, h, color in crops:
        rect = patches.Rectangle(
            (x, y), w, h,
            linewidth=2, edgecolor=color, facecolor=color, alpha=0.3
        )
        ax.add_patch(rect)
        # Mark center
        cx, cy = x + w/2, y + h/2
        ax.plot(cx, cy, 'o', color=color, markersize=8)
        ax.annotate(name, (cx, cy), textcoords="offset points", 
                    xytext=(5, 5), fontsize=9, color=color, fontweight='bold')
    
    if title:
        ax.set_title(title, fontsize=11, fontweight='bold')

## 1. Independent Crops (Different Seeds)

When crops have **different seeds**, they sample independently and end up in unrelated regions.

In [ ]:
# Image dimensions
W, H = 384, 256
ratio = (1.0, 1.0)  # Square crops for clarity

# Independent crops with different seeds
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, seed_base in enumerate([42, 100, 200]):
    # Different seeds for each crop
    x1, y1, w1, h1 = get_crop_params(W, H, (0.4, 1.0), ratio, seed=seed_base)
    x2, y2, w2, h2 = get_crop_params(W, H, (0.1, 0.4), ratio, seed=seed_base + 50)  # Different seed!
    
    crops = [
        ('large', x1, y1, w1, h1, 'blue'),
        ('small', x2, y2, w2, h2, 'red'),
    ]
    plot_crop_regions(axes[idx], W, H, crops, f'Independent (seeds {seed_base}, {seed_base+50})')

fig.suptitle('Independent Crops: Different seeds produce unrelated regions', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 2. Yoked Crops (Same Seed)

When crops have the **same seed**, they share the random number sequence. With different scale ranges, this produces a **zoom pair** - crops centered on the same region but at different zoom levels.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, seed in enumerate([42, 100, 200]):
    # SAME seed for both crops (yoked)
    x1, y1, w1, h1 = get_crop_params(W, H, (0.4, 1.0), ratio, seed=seed)
    x2, y2, w2, h2 = get_crop_params(W, H, (0.1, 0.4), ratio, seed=seed)  # Same seed!
    
    crops = [
        ('large (0.4-1.0)', x1, y1, w1, h1, 'blue'),
        ('small (0.1-0.4)', x2, y2, w2, h2, 'red'),
    ]
    plot_crop_regions(axes[idx], W, H, crops, f'Yoked (seed={seed})')

fig.suptitle('Yoked Crops: Same seed + different scales = zoom pairs', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 3. Standard vs Direct RRC

Both `crop_mode="standard"` (torchvision-style rejection sampling) and `crop_mode="direct"` (analytic) support yoked crops, but they produce slightly different distributions.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for row, (mode, mode_label) in enumerate([('standard', 'Standard RRC'), ('direct', 'Direct RRC')]):
    for idx, seed in enumerate([42, 100, 200]):
        x1, y1, w1, h1 = get_crop_params(W, H, (0.4, 1.0), ratio, seed=seed, crop_mode=mode)
        x2, y2, w2, h2 = get_crop_params(W, H, (0.1, 0.4), ratio, seed=seed, crop_mode=mode)
        
        crops = [
            ('large', x1, y1, w1, h1, 'blue'),
            ('small', x2, y2, w2, h2, 'red'),
        ]
        plot_crop_regions(axes[row][idx], W, H, crops, f'{mode_label} (seed={seed})')

fig.suptitle('Yoked Crops: Standard vs Direct RRC', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 4. Effect of Scale Ranges

The relationship between yoked crops depends on their scale ranges. Here we explore different configurations:

In [ ]:
# Different scale range configurations
configs = [
    # (large_scale, small_scale, description)
    ((0.6, 1.0), (0.1, 0.3), 'Wide separation'),
    ((0.4, 0.8), (0.2, 0.5), 'Overlapping ranges'),
    ((0.5, 1.0), (0.05, 0.2), 'Extreme zoom'),
]

fig, axes = plt.subplots(len(configs), 3, figsize=(15, 5 * len(configs)))
seed = 42

for row, (large_scale, small_scale, desc) in enumerate(configs):
    for col, seed in enumerate([42, 100, 200]):
        x1, y1, w1, h1 = get_crop_params(W, H, large_scale, ratio, seed=seed)
        x2, y2, w2, h2 = get_crop_params(W, H, small_scale, ratio, seed=seed)
        
        crops = [
            (f'large {large_scale}', x1, y1, w1, h1, 'blue'),
            (f'small {small_scale}', x2, y2, w2, h2, 'red'),
        ]
        title = f'{desc}\nseed={seed}' if col == 1 else f'seed={seed}'
        plot_crop_regions(axes[row][col], W, H, crops, title)
    axes[row][0].set_ylabel(desc, fontsize=12, fontweight='bold')

fig.suptitle('Effect of Scale Ranges on Yoked Crops', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

## 5. Non-Square Aspect Ratios

With non-square aspect ratios, yoked crops still share the same center but may have different shapes.

In [ ]:
# Standard torchvision ratio range
ratio_wide = (3/4, 4/3)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, seed in enumerate([42, 100, 200]):
    x1, y1, w1, h1 = get_crop_params(W, H, (0.4, 1.0), ratio_wide, seed=seed)
    x2, y2, w2, h2 = get_crop_params(W, H, (0.1, 0.4), ratio_wide, seed=seed)
    
    crops = [
        ('large', x1, y1, w1, h1, 'blue'),
        ('small', x2, y2, w2, h2, 'red'),
    ]
    plot_crop_regions(axes[idx], W, H, crops, f'Yoked (seed={seed}, ratio=3/4-4/3)')

fig.suptitle('Yoked Crops with Variable Aspect Ratio (3/4 to 4/3)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 6. Multiple Yoked Pairs

You can have multiple yoked pairs in the same batch. Each pair shares its own seed.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 7))

# Two yoked pairs with different seeds
seed_pair1 = 42
seed_pair2 = 99

x1, y1, w1, h1 = get_crop_params(W, H, (0.4, 1.0), ratio, seed=seed_pair1)
x2, y2, w2, h2 = get_crop_params(W, H, (0.1, 0.4), ratio, seed=seed_pair1)
x3, y3, w3, h3 = get_crop_params(W, H, (0.4, 1.0), ratio, seed=seed_pair2)
x4, y4, w4, h4 = get_crop_params(W, H, (0.1, 0.4), ratio, seed=seed_pair2)

crops = [
    ('pair1_large', x1, y1, w1, h1, 'blue'),
    ('pair1_small', x2, y2, w2, h2, 'dodgerblue'),
    ('pair2_large', x3, y3, w3, h3, 'red'),
    ('pair2_small', x4, y4, w4, h4, 'orange'),
]
plot_crop_regions(ax, W, H, crops, 'Two Yoked Pairs (blue pair, red pair)')

plt.tight_layout()
plt.show()

print("Blue pair (seed=42): large and small crops share the same center")
print("Red pair (seed=99): large and small crops share the same center")
print("The two pairs are independent of each other.")

## 7. Usage in MultiRandomResizedCrop

In practice, yoked crops are specified by giving crops the same `seed` parameter:

In [ ]:
from slipstream.decoders import MultiRandomResizedCrop

# Yoked crops: same seed produces zoom pairs
yoked_config = {
    "zoom_out": dict(size=224, scale=(0.4, 1.0), ratio=(1.0, 1.0), seed=42),
    "zoom_in":  dict(size=224, scale=(0.1, 0.4), ratio=(1.0, 1.0), seed=42),  # Same seed!
}

# Independent crops: different seeds
independent_config = {
    "crop_a": dict(size=224, scale=(0.4, 1.0), ratio=(1.0, 1.0), seed=42),
    "crop_b": dict(size=224, scale=(0.1, 0.4), ratio=(1.0, 1.0), seed=99),  # Different seed!
}

print("Yoked configuration (same seed=42):")
for name, params in yoked_config.items():
    print(f"  {name}: scale={params['scale']}, seed={params['seed']}")

print("\nIndependent configuration (different seeds):")
for name, params in independent_config.items():
    print(f"  {name}: scale={params['scale']}, seed={params['seed']}")

## Summary

| Configuration | Behavior |
|--------------|----------|
| **Independent** (different seeds) | Crops sample independently, unrelated regions |
| **Yoked** (same seed) | Crops share random sequence, same center point |
| **Yoked + different scales** | Zoom pairs: same center, different zoom levels |

**Use cases for yoked crops:**
- Multi-scale consistency learning
- Forcing views to see the same object at different resolutions
- Zoom-in/zoom-out augmentation pairs